In [8]:
import matplotlib.pyplot as plt
from astropy.io import fits
import numpy as np
import scipy.ndimage
from scipy import fftpack
import time

t0 = time.time()

In [9]:
def SNR(powerspecs):
    #Calculate Signal-to-Noise ratio given a set of powerspectra 
    
    powermean = np.mat(np.mean(powerspecs, axis = 0))
    powermat = np.vstack((powerspecs))
    covar = np.mat(np.cov(powerspecs, rowvar = 0))
    
    SNRsquare = powermean * (covar.I * powermean.T)
    
    return np.sqrt(SNRsquare)

In [10]:
def power1D(image, num_bins):
    
    y, x = np.indices(image.shape)
    center = np.array([(x.max() - x.min()) / 2., (x.max() - x.min()) / 2.])
    
    if image.shape[0] % 2 == 0:
        center += 0.5
    
    radii = np.hypot(x - center[0], y - center[1])
    
    sorted_radii_indices = np.argsort(radii.flat)
    sorted_radii = radii.flat[sorted_radii_indices]
    sorted_pixels = image.flat[sorted_radii_indices]
    
    bins = np.logspace(0, np.log10(image.shape[0]/2.), num_bins + 1)
    
    bin_weights = np.histogram(sorted_radii, bins)[0]
    bin_edges = np.cumsum(bin_weights)
    pixel_sums = np.cumsum(sorted_pixels, dtype=float)
    bin_totals = pixel_sums[bin_edges[1:] - 1] - pixel_sums[bin_edges[:-1] - 1]
    radial_prof = bin_totals/bin_weights[1:]
    
    return bins[1:], radial_prof

def PowerSpectrum(psd2D, sizedeg = 12.25, size = 2048, bins = 50):
    
    ells, psd1D = power1D(psd2D, num_bins = 50)
    
    edge2center = lambda x: x[:-1]+0.5*(x[1:]-x[:-1])
    ells = edge2center(ells)
    
    ells *= 360. / np.sqrt(sizedeg)
    norm = ((2 * np.pi * np.sqrt(sizedeg) / 360.0) ** 2) / (size ** 2) ** 2
    powspec = ells * (ells + 1) / (2 * np.pi) * norm * psd1D
    
    last_nan = np.where(np.isnan(powspec))[0][-1]
    ells = ells[last_nan + 1:]
    powspec = powspec[last_nan + 1:]
    return ells, powspec

In [11]:
image_1 = fits.open('file:///C:/cygwin64/home/rehg98/WLconv_z1100.00_0580r.fits')[0].data.astype(float)
image_1 = scipy.ndimage.filters.gaussian_filter(image_1, 9.75)
F_1 = fftpack.fftshift(fftpack.fft2(image_1))
psd2D_1 = np.abs(F_1)**2
ell_1, powspec_1 = PowerSpectrum(psd2D_1, sizedeg = 12.25, size = 2048, bins = 50)

C:\Users\rehg98\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide


In [12]:
image_2 = fits.open('file:///C:/cygwin64/home/rehg98/WLconv_z1100.00_0007r.fits')[0].data.astype(float)
image_2 = scipy.ndimage.filters.gaussian_filter(image_2, 9.75)
F_2 = fftpack.fftshift(fftpack.fft2(image_2))
psd2D_2 = np.abs(F_2)**2
ell_2, powspec_2 = PowerSpectrum(psd2D_2, sizedeg = 12.25, size = 2048, bins = 50)

C:\Users\rehg98\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide


In [13]:
#plt.loglog(ell_1, powspec_1)
#plt.show()

#plt.loglog(ell_2, powspec_2)
#plt.show()

In [14]:
powspecs = np.array([powspec_1, powspec_2])
print(SNR(powspecs))

t1 = time.time()
print(t1 - t0)

[[  1.61798959e+10]]
3.9927473068237305
